## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# 💬 Azure AI Agent with Thread Management - Multi-Turn Conversations 🏦

This notebook demonstrates working with conversation threads using `AzureAIProjectAgentProvider` for multi-turn conversations like loan application discussions and financial planning sessions.

## Features Covered:
- Creating and managing persistent conversation threads
- Using `agent.get_new_thread()` for thread management
- Thread lifecycle management (create, use, delete)
- Multi-turn conversation continuity for banking discussions
- Thread initialization and validation with `store=False` option

### ⚠️ Important Note ⚠️
> **Threads maintain conversation context across multiple interactions. This is essential for complex banking discussions like loan applications or investment planning.**

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Microsoft Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with:
   - `AI_FOUNDRY_PROJECT_ENDPOINT`
   - `AZURE_AI_MODEL_DEPLOYMENT_NAME`
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant:
```bash
az login --tenant <tenant-id>
```

This example demonstrates how to work with existing threads to maintain conversation continuity in banking scenarios.

## Import Libraries

Import the required libraries using the `AzureAIAgentsProvider` API:

In [ ]:
import os
from pathlib import Path
from random import randint, uniform
from typing import Annotated

from agent_framework.azure import AzureAIAgentsProvider
from azure.ai.agents.aio import AgentsClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field
from dotenv import load_dotenv

# Load environment variables
load_dotenv('../../.env')

# Verify environment setup
endpoint = os.getenv('AI_FOUNDRY_PROJECT_ENDPOINT')
model = os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME')

print("🔧 Environment Configuration:")
print(f"✅ Project Endpoint: {endpoint[:50]}..." if endpoint else "❌ AI_FOUNDRY_PROJECT_ENDPOINT not set")
print(f"✅ Model Deployment: {model}" if model else "❌ AZURE_AI_MODEL_DEPLOYMENT_NAME not set")

## Check Environment Variables

Verify that the required environment variables are set:

In [ ]:
# Check required environment variables
endpoint = os.getenv('AI_FOUNDRY_PROJECT_ENDPOINT')
if endpoint:
    print(f"✅ AI_FOUNDRY_PROJECT_ENDPOINT: {endpoint}")
else:
    print("❌ AI_FOUNDRY_PROJECT_ENDPOINT: Not set")
    print("⚠️  Please set the AI_FOUNDRY_PROJECT_ENDPOINT environment variable")

## Define Function Tools 🏦

Let's define banking functions for our loan application discussions:

In [ ]:
def get_loan_prequalification(
    loan_amount: Annotated[float, Field(description="Requested loan amount in dollars.")],
    credit_score: Annotated[int, Field(description="Customer's credit score.")],
    annual_income: Annotated[float, Field(description="Customer's annual income in dollars.")],
) -> str:
    """Check loan prequalification status based on customer data."""
    # Simple prequalification logic for demo
    dti = loan_amount / (annual_income * 30)  # Simplified debt-to-income estimate
    
    if credit_score >= 700 and dti < 0.43:
        status = "Pre-Qualified ✅"
        rate = "5.99% - 6.75%"
    elif credit_score >= 650 and dti < 0.50:
        status = "Conditionally Pre-Qualified ⚠️"
        rate = "7.25% - 8.50%"
    else:
        status = "Additional Review Required 📋"
        rate = "8.50% - 12.00%"
    
    return f"""Loan Pre-Qualification Result:
    Requested Amount: ${loan_amount:,.2f}
    Credit Score: {credit_score}
    Annual Income: ${annual_income:,.2f}
    Status: {status}
    Estimated Rate Range: {rate}"""


def get_required_documents(
    loan_type: Annotated[str, Field(description="Type of loan: mortgage, auto, personal")],
) -> str:
    """Get list of required documents for loan application."""
    docs = {
        "mortgage": [
            "Government-issued ID",
            "Pay stubs (last 30 days)",
            "W-2 forms (last 2 years)",
            "Tax returns (last 2 years)",
            "Bank statements (last 2 months)",
            "Proof of assets",
            "Property information"
        ],
        "auto": [
            "Government-issued ID",
            "Proof of income",
            "Proof of insurance",
            "Vehicle information (VIN, mileage)"
        ],
        "personal": [
            "Government-issued ID",
            "Proof of income",
            "Proof of residence"
        ]
    }
    loan_type = loan_type.lower()
    if loan_type in docs:
        doc_list = "\n    - ".join(docs[loan_type])
        return f"Required Documents for {loan_type.title()} Loan:\n    - {doc_list}"
    return "Please specify: mortgage, auto, or personal loan"

## Create and Use Existing Thread 💬

This example shows how to:
1. Create a thread that persists for the conversation
2. Use `agent.get_new_thread(service_thread_id=...)` to continue conversations
3. Properly clean up thread resources

**Key API Methods**: 
- `agent.get_new_thread()` - Creates new thread (with optional `store=False`)
- `agent.get_new_thread(service_thread_id=...)` - Uses existing thread

In [ ]:
async def main() -> None:
    print("=== 💬 Azure AI Agent with Thread Management - Loan Application ===")

    async with (
        AzureCliCredential() as credential,
        AgentsClient(
            endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"],
            credential=credential
        ) as agents_client,
        AzureAIAgentsProvider(agents_client=agents_client) as provider,
    ):
        # Create a thread that will persist for the loan discussion
        created_thread = await agents_client.threads.create()
        print(f"✅ Created conversation thread: {created_thread.id}")

        try:
            agent = await provider.create_agent(
                name="LoanApplicationAdvisor",
                instructions="""You are a Loan Application Advisor. Help customers through the loan application process.
                Remember context from earlier in the conversation to provide personalized guidance.
                Be professional and thorough in explaining requirements.""",
                tools=[get_loan_prequalification, get_required_documents],
            )
            
            # Use existing thread for conversation continuity
            thread = agent.get_new_thread(service_thread_id=created_thread.id)
            assert thread.is_initialized
            print(f"✅ Thread initialized: {thread.is_initialized}")
            print(f"   Thread ID: {thread.service_thread_id}")
            
            # First message in loan application discussion
            query = "I'm interested in applying for a mortgage. Can you check if I'd prequalify for $350,000 with a credit score of 720 and annual income of $95,000?"
            print(f"\n🤔 Customer: {query}")
            result = await agent.run(query, thread=thread)
            print(f"🏦 Advisor: {result}")
            
        finally:
            # Clean up the thread
            await agents_client.threads.delete(created_thread.id)
            print(f"\n🗑️ Deleted thread: {created_thread.id}")

## 🚀 Execute the Initial Loan Discussion

In [ ]:
await main()

## 🔄 Multi-Turn Loan Application Flow with Thread Continuity

In [ ]:
async def multi_turn_loan_application() -> None:
    """
    Demonstrates a complete multi-turn loan application conversation
    where thread context maintains the discussion history.
    """
    print("=== 💬 Multi-Turn Loan Application Workflow ===")
    
    async with (
        AzureCliCredential() as credential,
        AgentsClient(
            endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"],
            credential=credential
        ) as agents_client,
        AzureAIAgentsProvider(agents_client=agents_client) as provider,
    ):
        # Create persistent thread for entire loan application journey
        thread_info = await agents_client.threads.create()
        print(f"✅ Started loan application session: {thread_info.id}")
        
        try:
            advisor = await provider.create_agent(
                name="LoanApplicationAdvisor",
                instructions="""You are a comprehensive Loan Application Advisor. Guide customers through:
                1. Understanding loan options
                2. Prequalification assessment  
                3. Document requirements
                4. Next steps in the application process
                
                Remember the conversation context and reference earlier information when helpful.
                Be encouraging while being realistic about qualifications.""",
                tools=[get_loan_prequalification, get_required_documents],
            )
            
            # Use persistent thread
            thread = advisor.get_new_thread(service_thread_id=thread_info.id)
            
            # Loan application conversation flow
            conversation = [
                "Hi, I'm looking to buy my first home and need information about mortgage options.",
                "Can you check if I'd prequalify? My credit score is 680, annual income is $72,000, and I'm looking at homes around $280,000.",
                "Based on my prequalification status, what documents would I need for a conventional mortgage?",
                "Given my situation, would you recommend I improve my credit score first before applying?"
            ]
            
            for i, message in enumerate(conversation, 1):
                print(f"\n📩 Turn {i} - Customer: {message}")
                response = await advisor.run(message, thread=thread)
                print(f"🏦 Advisor: {response}")
                print("-" * 50)
                
        finally:
            await agents_client.threads.delete(thread_info.id)
            print(f"\n✅ Loan application session completed and cleaned up")

## 🚀 Execute Multi-Turn Loan Application

In [ ]:
await multi_turn_loan_application()

## 📝 Key Takeaways

### Thread Management in Applications

1. **Thread Persistence**: Threads maintain conversation context across multiple interactions, essential for complex multi-turn discussions

2. **Creating Threads**: Use `AgentsClient` to create threads before agent interactions:
   ```python
   created_thread = await agents_client.threads.create()
   ```

3. **Using Existing Threads**: Connect agent to existing thread for conversation continuity:
   ```python
   thread = agent.get_new_thread(service_thread_id=created_thread.id)
   ```

4. **Thread Lifecycle**: Always clean up threads when conversation is complete:
   ```python
   await agents_client.threads.delete(thread_id)
   ```

### Use Case Benefits

- **Compliance**: Persistent threads create audit trails for regulatory compliance
- **Customer Experience**: Customers don't need to repeat information
- **Context-Aware Responses**: Agents can reference earlier parts of the conversation
- **Multi-Step Processes**: Perfect for loan applications, account onboarding, and consultations

### ⚠️ Disclaimer
This example uses simulated data for demonstration purposes. In production:
- Connect to actual backend systems
- Implement proper authentication and authorization
- Follow all regulatory requirements
- Ensure compliance for sensitive data